In [1]:
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Load the dataset
digits = datasets.load_digits()
X = digits.data
y = digits.target

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Evaluate clustering performance using Silhouette Coefficient
def evaluate_silhouette(X, labels):
    unique_labels = set(labels)
    # Exclude cases with no clusters or only one cluster
    if len(unique_labels) <= 1 or (len(unique_labels) == 2 and -1 in unique_labels):
        return -1  # Return -1 for undefined silhouette
    # Exclude noise points (-1 labels) for DBSCAN
    mask = labels != -1
    return silhouette_score(X[mask], labels[mask])

# 1. K-Means Clustering
print("K-Means Clustering:")
kmeans_params = {'n_clusters': range(8, 12), 'random_state': [42]}  # Search for optimal n_clusters
best_kmeans = None
best_score = -1

for n_clusters in kmeans_params['n_clusters']:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X_scaled)
    silhouette = evaluate_silhouette(X_scaled, kmeans.labels_)
    print(f"n_clusters={n_clusters}, Silhouette Coefficient={silhouette:.4f}")
    if silhouette > best_score:
        best_score = silhouette
        best_kmeans = kmeans

print(f"Best K-Means Silhouette Coefficient: {best_score:.4f}\n")

# 2. DBSCAN Clustering
print("DBSCAN Clustering:")
dbscan_params = {'eps': [1.5, 2.0, 2.5], 'min_samples': [3, 5, 10]}
best_dbscan = None
best_score = -1

for eps in dbscan_params['eps']:
    for min_samples in dbscan_params['min_samples']:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan.fit(X_scaled)
        labels = dbscan.labels_
        silhouette = evaluate_silhouette(X_scaled, labels)
        if silhouette != -1:  # Only print results if silhouette is defined
            print(f"eps={eps}, min_samples={min_samples}, Silhouette Coefficient={silhouette:.4f}")
        else:
            print(f"eps={eps}, min_samples={min_samples}, Silhouette Coefficient=Undefined (no clusters)")
        if silhouette > best_score:
            best_score = silhouette
            best_dbscan = dbscan

print(f"Best DBSCAN Silhouette Coefficient: {best_score:.4f}\n")

# Final Results Summary
print("Final Results:")
if best_kmeans:
    print(f"K-Means Best Silhouette Coefficient: {evaluate_silhouette(X_scaled, best_kmeans.labels_):.4f}")
if best_dbscan:
    print(f"DBSCAN Best Silhouette Coefficient: {evaluate_silhouette(X_scaled, best_dbscan.labels_):.4f}")

K-Means Clustering:
n_clusters=8, Silhouette Coefficient=0.1274
n_clusters=9, Silhouette Coefficient=0.1301
n_clusters=10, Silhouette Coefficient=0.1356
n_clusters=11, Silhouette Coefficient=0.1483
Best K-Means Silhouette Coefficient: 0.1483

DBSCAN Clustering:
eps=1.5, min_samples=3, Silhouette Coefficient=Undefined (no clusters)
eps=1.5, min_samples=5, Silhouette Coefficient=Undefined (no clusters)
eps=1.5, min_samples=10, Silhouette Coefficient=Undefined (no clusters)
eps=2.0, min_samples=3, Silhouette Coefficient=0.2905
eps=2.0, min_samples=5, Silhouette Coefficient=0.2391
eps=2.0, min_samples=10, Silhouette Coefficient=Undefined (no clusters)
eps=2.5, min_samples=3, Silhouette Coefficient=0.4522
eps=2.5, min_samples=5, Silhouette Coefficient=0.6787
eps=2.5, min_samples=10, Silhouette Coefficient=Undefined (no clusters)
Best DBSCAN Silhouette Coefficient: 0.6787

Final Results:
K-Means Best Silhouette Coefficient: 0.1483
DBSCAN Best Silhouette Coefficient: 0.6787


This code performs clustering on the hand-written digits dataset using two algorithms: K-Means and DBSCAN, evaluating their performance using the Silhouette Coefficient. First, it standardizes the dataset to ensure consistent scaling of features. It then runs K-Means clustering for different values of n_clusters (from 8 to 11). Then the Silhouette Coefficient is calculated to assess how well-separated the clusters are. The best K-Means model is selected based on the highest Silhouette score. Similarly, the code performs DBSCAN clustering by testing different combinations of eps (the maximum distance between points) and min_samples (the minimum number of points required to form a cluster), again calculating the Silhouette Coefficient for each combination. The best DBSCAN model is chosen based on the highest Silhouette score. Finally, the code outputs the best Silhouette Coefficients for both K-Means and DBSCAN.

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.optimize import linear_sum_assignment  # Corrected import

# Load the dataset
digits = datasets.load_digits()
X = digits.data
y = digits.target

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Evaluate clustering performance using Silhouette Coefficient
def evaluate_silhouette(X, labels):
    unique_labels = set(labels)
    if len(unique_labels) <= 1 or (len(unique_labels) == 2 and -1 in unique_labels):
        return -1  # Return -1 for undefined silhouette
    # Exclude noise points (-1 labels) for DBSCAN
    mask = labels != -1
    return silhouette_score(X[mask], labels[mask])

# Calculate accuracy between predicted labels and true labels
def calculate_accuracy(true_labels, predicted_labels):
    # Construct confusion matrix
    n_clusters = len(np.unique(predicted_labels))
    cost_matrix = np.zeros((n_clusters, 10))  # 10 digits (classes)

    for cluster_id in range(n_clusters):
        for true_id in range(10):  # For each digit
            cost_matrix[cluster_id, true_id] = np.sum((predicted_labels == cluster_id) & (true_labels == true_id))

    # Handle cases with no valid clusters (e.g., empty clusters or no valid matches)
    if np.sum(cost_matrix) == 0:
        return 0.0  # Return 0 if there are no valid matches

    # Use Hungarian algorithm (linear_sum_assignment) to find the optimal assignment
    row_ind, col_ind = linear_sum_assignment(-cost_matrix)  # Maximize the match by negating the cost matrix
    if len(true_labels) == 0:  # Avoid division by zero if there are no true labels
        return 0.0

    accuracy = cost_matrix[row_ind, col_ind].sum() / len(true_labels)
    return accuracy

# 1. K-Means Clustering
print("K-Means Clustering:")
kmeans_params = {'n_clusters': range(8, 12), 'random_state': [42]}  # Search for optimal n_clusters
best_kmeans = None
best_kmeans_score = -1
best_kmeans_accuracy = -1

for n_clusters in kmeans_params['n_clusters']:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X_scaled)
    silhouette = evaluate_silhouette(X_scaled, kmeans.labels_)
    kmeans_accuracy = calculate_accuracy(y, kmeans.labels_)  # Calculate accuracy for K-Means
    print(f"n_clusters={n_clusters}, Silhouette Coefficient={silhouette:.4f}, Accuracy={kmeans_accuracy:.4f}")
    if silhouette > best_kmeans_score:
        best_kmeans_score = silhouette
        best_kmeans = kmeans
        best_kmeans_accuracy = kmeans_accuracy

print(f"Best K-Means Silhouette Coefficient: {best_kmeans_score:.4f}, Accuracy: {best_kmeans_accuracy:.4f}\n")

# 2. DBSCAN Clustering
print("DBSCAN Clustering:")
dbscan_params = {'eps': [1.5, 2.0, 2.5], 'min_samples': [3, 5, 10]}
best_dbscan = None
best_dbscan_score = -1
best_dbscan_accuracy = -1

for eps in dbscan_params['eps']:
    for min_samples in dbscan_params['min_samples']:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan.fit(X_scaled)
        labels = dbscan.labels_
        silhouette = evaluate_silhouette(X_scaled, labels)
        dbscan_accuracy = calculate_accuracy(y[labels != -1], labels[labels != -1])  # Ignore noise points (-1)
        if silhouette != -1:  # Only print results if silhouette is defined
            print(f"eps={eps}, min_samples={min_samples}, Silhouette Coefficient={silhouette:.4f}, Accuracy={dbscan_accuracy:.4f}")
        else:
            print(f"eps={eps}, min_samples={min_samples}, Silhouette Coefficient=Undefined (no clusters)")
        if silhouette > best_dbscan_score:
            best_dbscan_score = silhouette
            best_dbscan = dbscan
            best_dbscan_accuracy = dbscan_accuracy

print(f"Best DBSCAN Silhouette Coefficient: {best_dbscan_score:.4f}, Accuracy: {best_dbscan_accuracy:.4f}\n")

# Final Results Summary
print("Final Results:")
if best_kmeans:
    print(f"K-Means Best Silhouette Coefficient: {evaluate_silhouette(X_scaled, best_kmeans.labels_):.4f}, Accuracy: {best_kmeans_accuracy:.4f}")
if best_dbscan:
    print(f"DBSCAN Best Silhouette Coefficient: {evaluate_silhouette(X_scaled, best_dbscan.labels_):.4f}, Accuracy: {best_dbscan_accuracy:.4f}")

K-Means Clustering:
n_clusters=8, Silhouette Coefficient=0.1274, Accuracy=0.6210
n_clusters=9, Silhouette Coefficient=0.1301, Accuracy=0.6238
n_clusters=10, Silhouette Coefficient=0.1356, Accuracy=0.6661
n_clusters=11, Silhouette Coefficient=0.1483, Accuracy=0.6967
Best K-Means Silhouette Coefficient: 0.1483, Accuracy: 0.6967

DBSCAN Clustering:
eps=1.5, min_samples=3, Silhouette Coefficient=Undefined (no clusters)
eps=1.5, min_samples=5, Silhouette Coefficient=Undefined (no clusters)
eps=1.5, min_samples=10, Silhouette Coefficient=Undefined (no clusters)
eps=2.0, min_samples=3, Silhouette Coefficient=0.2905, Accuracy=0.7200
eps=2.0, min_samples=5, Silhouette Coefficient=0.2391, Accuracy=0.5263
eps=2.0, min_samples=10, Silhouette Coefficient=Undefined (no clusters)
eps=2.5, min_samples=3, Silhouette Coefficient=0.4522, Accuracy=0.7558
eps=2.5, min_samples=5, Silhouette Coefficient=0.6787, Accuracy=1.0000
eps=2.5, min_samples=10, Silhouette Coefficient=Undefined (no clusters)
Best DBSCA

The code performs clustering on the Digits dataset using K-Means and DBSCAN algorithms. It evaluates clustering performance based on the Silhouette Coefficient and accuracy. The accuracy is calculated to match predicted clusters to true labels. K-Means searches for the optimal number of clusters between 8 and 11, while DBSCAN tests different values of eps and min_samples. The best clustering method is determined by comparing the highest Silhouette Coefficient and accuracy achieved. Experiment is performed using different number of clusters for k-means and different values of eps and min_samples for dbscan, out of that for k-means, Best Silhouette Coefficient: 0.1483, Accuracy: 0.6967 and for DBSCAN, Best Silhouette Coefficient: 0.6787, Accuracy: 1.0000. Though Dbscan also leaves noise points (labeled as -1) and may produce fewer valid clusters or no cluster, leads to lower accuracy in some cases. If we ignore this no cluster points then For this Digit dataset Dbscan peforms better with higher Silhouette Coefficient: 0.6787 compared to k-means which have best Silhouette Coefficient: 0.1483.